# Serialization of Phonopy Objects and Custom JSON Decoder 
Is it possible to transform Phonopy object into a json string easily?

In [1]:
import pickle
import json
from pprint import pprint
import numpy as np
import phonopy 

In [2]:
with open('phonon.pick', 'rb') as f:
    phonon = pickle.load(f)

In [3]:
pd = phonon.__dict__

In [4]:
phonon._supercell.__dict__

{'_is_old_style': 1e-05,
 '_s2u_map': array([0, 0, 0, 0, 4, 4, 4, 4]),
 '_u2s_map': array([0, 4]),
 '_u2u_map': {0: 0, 4: 1},
 '_supercell_matrix': array([[-1,  1,  1],
        [ 1, -1,  1],
        [ 1,  1, -1]], dtype=int32),
 'cell': array([[ 5.42606753e+00, -2.57389089e-18,  0.00000000e+00],
        [-2.57389089e-18,  5.42606753e+00,  0.00000000e+00],
        [ 2.57389089e-18,  2.57389089e-18,  5.42606753e+00]]),
 'scaled_positions': array([[0.  , 0.  , 0.  ],
        [1.  , 0.5 , 0.5 ],
        [0.5 , 0.  , 0.5 ],
        [0.5 , 0.5 , 0.  ],
        [0.25, 0.25, 0.25],
        [0.25, 0.75, 0.75],
        [0.75, 0.25, 0.75],
        [0.75, 0.75, 0.25]]),
 'symbols': ['Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si'],
 'numbers': array([14, 14, 14, 14, 14, 14, 14, 14], dtype=int32),
 'masses': array([28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085]),
 'magmoms': None}

In [5]:
def get_dict(obj):
    """resolve nested object recursively"""
    try:
        dic = obj.__dict__
    except AttributeError:
        return obj
            
    for key in dic.keys():
        if hasattr(dic[key], '__dict__'):
            dic[key] = get_dict(dic[key])
        if isinstance(dic[key], list):
            temp = [get_dict(d) for d in dic[key]]
            dic[key] = temp
    return dic    

In [6]:
def stringify_keys(d):
    """Convert a dict's keys to strings if they are not."""
    for key in d.keys():

        # check inner dict
        if isinstance(d[key], dict):
            value = stringify_keys(d[key])
        else:
            value = d[key]

        # convert nonstring to string if needed
        if not isinstance(key, str):
            try:
                d[str(key)] = value
            except Exception:
                try:
                    d[repr(key)] = value
                except Exception:
                    raise

            # delete old key
            del d[key]
    return d

In [7]:
dp = stringify_keys(get_dict(phonon))

## Custom JSON Encoding/Decoding

In [8]:
class NumpyEncoder(json.JSONEncoder):
    """ Decode numerical objects that json cannot parse by default"""
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, (np.int32, np.int64)):
            return int(obj)
        if isinstance(obj, complex):
            return (float(obj.real), float(obj.imag))
        return super().default(self, obj)

In [9]:
try:
    json.dumps(pd, cls=NumpyEncoder)
except Exception as inst:
    print('Exception occured: ', inst)

### Complex part

In [10]:
z=3+4j
jz = json.dumps(z, cls=NumpyEncoder)
jz

'[3.0, 4.0]'

In [11]:
json.loads(jz)

[3.0, 4.0]

### Modifiy NumpyEncoder to provide metadata for complex numbers

In [12]:
class NumpyEncoder(json.JSONEncoder):
    """ Decode numerical objects that json cannot parse by default"""
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, (np.int32, np.int64)):
            return int(obj)
        if isinstance(obj, complex):
            return {"__complex__": True,
                    "Re": obj.real,
                    "Im": obj.imag}
        return super().default(self, obj)

In [13]:
z=3+4j
jz = json.dumps(z, cls=NumpyEncoder)
jz

'{"__complex__": true, "Re": 3.0, "Im": 4.0}'

In [14]:
json.loads(jz)

{'__complex__': True, 'Re': 3.0, 'Im': 4.0}

In [15]:
def decode_complex(dct):
    if "__complex__" in dct:
        return complex(dct['Re'], dct['Im'])
    return dct

In [16]:
json.loads(jz, object_hook=decode_complex)

(3+4j)

In [17]:
with open('phonon.json', 'w') as f:
    json.dump(pd, f, cls=NumpyEncoder)